In [1]:
import pandas as pd
import numpy as np
import pprint
import re
import random
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
init_health = pd.read_csv('../ADNI/Medical_History/INITHEALTH.csv',low_memory=False)
rec_hist = pd.read_csv('../ADNI/Medical_History/RECMHIST.csv',low_memory=False)

In [3]:
def find_word_match(prev_dx):
    regex = r"diab|glucose intolerance"
    index_list = []
    for w in range(len(prev_dx)):
        try:
            matches = re.findall(regex, prev_dx[w], re.MULTILINE | re.IGNORECASE)
            if len(matches)!=0:
                index_list.append(w)
        except Exception:
            print(prev_dx[w])
    
    return index_list

In [4]:
def null_count(df):
    for i in df.columns:
        try:
            print(i,len([ x for x in df[i].tolist() if x=='']))
        except:
            print(i)

In [5]:
rec_hist_list = rec_hist.MHDESC.tolist()
rec_indices = find_word_match(rec_hist_list)
rec_hist  = rec_hist[rec_hist.index.isin(rec_indices)]
rid_rec_hist = rec_hist.RID.tolist()

init_health_list = init_health.IHDESC.tolist()
init_indices = find_word_match(init_health_list)
init_health  = init_health[init_health.index.isin(init_indices)]
rid_init_health = init_health.RID.tolist()


In [6]:
diabetes_rid_list = np.unique(rid_rec_hist+rid_init_health).tolist()

In [7]:
merge = pd.read_csv('../ADNI/Data___Database/ADNIMERGE.csv',low_memory=False)
select_col = ['RID', 'EXAMDATE','PTETHCAT', 'PTRACCAT', 'CDRSB', 'FDG', 'PIB', 'AV45', 'FBB',
              'Ventricles','Hippocampus','mPACCdigit','mPACCtrailsB']
merge = merge[select_col]
merge.EXAMDATE = pd.to_datetime(merge.EXAMDATE, dayfirst=True)
merge.isnull().sum()

RID                 0
EXAMDATE            0
PTETHCAT            0
PTRACCAT            0
CDRSB            4542
FDG             12364
PIB             15834
AV45            12977
FBB             15542
Ventricles       6747
Hippocampus      7410
mPACCdigit       4791
mPACCtrailsB     4785
dtype: int64

In [8]:
merge.PTETHCAT.value_counts()

Not Hisp/Latino    15460
Hisp/Latino          525
Unknown               72
Name: PTETHCAT, dtype: int64

In [9]:
merge.PTRACCAT.value_counts()

White                14804
Black                  719
Asian                  290
More than one          178
Unknown                 30
Am Indian/Alaskan       28
Hawaiian/Other PI        8
Name: PTRACCAT, dtype: int64

In [10]:
adni_merge = pd.read_excel('../ADNI/adni_merge.xlsx')
adni_merge

,RID,PTID,COLPROT,ORIGPROT,VISCODE,EXAMDATE,DX_bl,DX,AGE,MMSE,...,TAU_bl,PTAU_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting
0,2,011_S_0002,ADNI1,ADNI1,bl,2005-09-08,CN,CN,74.3,28.0,...,NaN,NaN,44,4,6,54.5455,44,4,6,54.5455
1,2,011_S_0002,ADNI1,ADNI1,m06,2006-03-06,CN,CN,74.3,28.0,...,NaN,NaN,44,4,6,54.5455,40,3,3,37.5000
2,2,011_S_0002,ADNI1,ADNI1,m36,2008-08-27,CN,CN,74.3,29.0,...,NaN,NaN,44,4,6,54.5455,34,2,7,87.5000
3,2,011_S_0002,ADNIGO,ADNI1,m60,2010-09-22,CN,CN,74.3,28.0,...,NaN,NaN,44,4,6,54.5455,37,5,4,40.0000
4,2,011_S_0002,ADNIGO,ADNI1,m66,2011-03-04,CN,CN,74.3,28.0,...,NaN,NaN,44,4,6,54.5455,37,5,4,40.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1784,7007,027_S_7007,ADNI3,ADNI3,bl,2021-10-18,LMCI,MCI,71.9,26.0,...,NaN,NaN,33,1,6,100.0000,33,1,6,100.0000
1785,7033,067_S_7033,ADNI3,ADNI3,bl,2022-02-01,AD,Dementia,59.4,26.0,...,NaN,NaN,28,4,7,100.0000,28,4,7,100.0000
1786,7062,021_S_7062,ADNI3,ADNI3,bl,2022-04-21,CN,CN,69.5,30.0,...,NaN,NaN,57,10,6,40.0000,57,10,6,40.0000
1787,7070,036_S_7070,ADNI3,ADNI3,bl,2022-05-04,LMCI,MCI,78.4,23.0,...,NaN,NaN,22,2,6,100.0000,22,2,6,100.0000


In [11]:
adni_merge.EXAMDATE = pd.to_datetime(adni_merge.EXAMDATE, dayfirst=True)
result = pd.merge(adni_merge, merge, how="left", on=["RID","EXAMDATE"])
result

,RID,PTID,COLPROT,ORIGPROT,VISCODE,EXAMDATE,DX_bl,DX,AGE,MMSE,...,PTRACCAT,CDRSB,FDG,PIB,AV45,FBB,Ventricles,Hippocampus,mPACCdigit,mPACCtrailsB
0,2,011_S_0002,ADNI1,ADNI1,bl,2005-09-08,CN,CN,74.3,28.0,...,White,0.0,1.25173,NaN,NaN,NaN,118233.0,8336.0,-4.31670,-4.11991
1,2,011_S_0002,ADNI1,ADNI1,m06,2006-03-06,CN,CN,74.3,28.0,...,White,0.0,1.33313,NaN,NaN,NaN,NaN,NaN,-6.71646,-6.01751
2,2,011_S_0002,ADNI1,ADNI1,m36,2008-08-27,CN,CN,74.3,29.0,...,White,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-2.63726,-2.64089
3,2,011_S_0002,ADNIGO,ADNI1,m60,2010-09-22,CN,CN,74.3,28.0,...,White,0.0,NaN,NaN,NaN,NaN,NaN,NaN,-4.16298,-5.01223
4,2,011_S_0002,ADNIGO,ADNI1,m66,2011-03-04,CN,CN,74.3,28.0,...,White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786,7007,027_S_7007,ADNI3,ADNI3,bl,2021-10-18,LMCI,MCI,71.9,26.0,...,White,1.5,NaN,NaN,NaN,1.3644,34064.2,5865.5,-11.18770,-7.70684
1787,7033,067_S_7033,ADNI3,ADNI3,bl,2022-02-01,AD,Dementia,59.4,26.0,...,Black,3.5,NaN,NaN,NaN,NaN,21393.3,5661.3,-14.27230,-10.25990
1788,7062,021_S_7062,ADNI3,ADNI3,bl,2022-04-21,CN,CN,69.5,30.0,...,Black,0.0,NaN,NaN,NaN,NaN,NaN,NaN,3.88952,3.86739
1789,7070,036_S_7070,ADNI3,ADNI3,bl,2022-05-04,LMCI,MCI,78.4,23.0,...,White,1.5,NaN,NaN,NaN,NaN,NaN,NaN,-17.78300,-13.61020


In [12]:
rid_set = set(adni_merge.RID.tolist())
result.isnull().sum()

RID                            0
PTID                           0
COLPROT                        0
ORIGPROT                       0
VISCODE                        0
EXAMDATE                       0
DX_bl                          0
DX                             0
AGE                            0
MMSE                           1
MMSE_bl                        1
PTGENDER                       0
PTEDUCAT                       0
APOE4                         71
ABETA                       1554
TAU                         1554
PTAU                        1554
ABETA_bl                     732
TAU_bl                       732
PTAU_bl                      732
RAVLT_immediate_bl             0
RAVLT_learning_bl              0
RAVLT_forgetting_bl            0
RAVLT_perc_forgetting_bl       0
RAVLT_immediate                0
RAVLT_learning                 0
RAVLT_forgetting               0
RAVLT_perc_forgetting          0
PTETHCAT                       0
PTRACCAT                       0
CDRSB     

In [13]:
collist = ['DX','AV45','MMSE','CDRSB','FDG','PIB','ABETA','TAU','PTAU','Ventricles','Hippocampus','mPACCdigit','mPACCtrailsB']
for x in collist:
    result[x] = result[x].fillna('')

In [14]:
def pulldown(df,col1):
    rid_row = df.RID.tolist()
    target_row = df[col1].tolist()
    n = len(rid_row)
    for i in range(1,n):
        if target_row[i]=='':
            if rid_row[i-1] == rid_row[i]:
                target_row[i] = target_row[i-1]
    return target_row

collist = ['DX','MMSE','CDRSB','FDG','PIB','ABETA','TAU','PTAU','Ventricles','Hippocampus','mPACCdigit','mPACCtrailsB']
for x in collist:
    result[x] = pulldown(result,x)
result

,RID,PTID,COLPROT,ORIGPROT,VISCODE,EXAMDATE,DX_bl,DX,AGE,MMSE,...,PTRACCAT,CDRSB,FDG,PIB,AV45,FBB,Ventricles,Hippocampus,mPACCdigit,mPACCtrailsB
0,2,011_S_0002,ADNI1,ADNI1,bl,2005-09-08,CN,CN,74.3,28.0,...,White,0.0,1.25173,,,NaN,118233.0,8336.0,-4.3167,-4.11991
1,2,011_S_0002,ADNI1,ADNI1,m06,2006-03-06,CN,CN,74.3,28.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-6.71646,-6.01751
2,2,011_S_0002,ADNI1,ADNI1,m36,2008-08-27,CN,CN,74.3,29.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-2.63726,-2.64089
3,2,011_S_0002,ADNIGO,ADNI1,m60,2010-09-22,CN,CN,74.3,28.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-4.16298,-5.01223
4,2,011_S_0002,ADNIGO,ADNI1,m66,2011-03-04,CN,CN,74.3,28.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-4.16298,-5.01223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786,7007,027_S_7007,ADNI3,ADNI3,bl,2021-10-18,LMCI,MCI,71.9,26.0,...,White,1.5,,,,1.3644,34064.2,5865.5,-11.1877,-7.70684
1787,7033,067_S_7033,ADNI3,ADNI3,bl,2022-02-01,AD,Dementia,59.4,26.0,...,Black,3.5,,,,NaN,21393.3,5661.3,-14.2723,-10.2599
1788,7062,021_S_7062,ADNI3,ADNI3,bl,2022-04-21,CN,CN,69.5,30.0,...,Black,0.0,,,,NaN,,,3.88952,3.86739
1789,7070,036_S_7070,ADNI3,ADNI3,bl,2022-05-04,LMCI,MCI,78.4,23.0,...,White,1.5,,,,NaN,,,-17.783,-13.6102


In [15]:
for id in rid_set:
    eg = result[result.RID==id]
    for i in range(len(eg)):
        diff = eg.EXAMDATE.iloc[i] - eg.EXAMDATE.iloc[0]
        eg['AGE'].iloc[i] =round(diff.days/365,1) + eg.AGE.iloc[0]
    result[result.RID==id] = eg

/var/folders/1q/y31g8gyd1njfgvn3tsd2xfjr0000gn/T/ipykernel_93704/2528748770.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eg['AGE'].iloc[i] =round(diff.days/365,1) + eg.AGE.iloc[0]


In [16]:
result

,RID,PTID,COLPROT,ORIGPROT,VISCODE,EXAMDATE,DX_bl,DX,AGE,MMSE,...,PTRACCAT,CDRSB,FDG,PIB,AV45,FBB,Ventricles,Hippocampus,mPACCdigit,mPACCtrailsB
0,2,011_S_0002,ADNI1,ADNI1,bl,2005-09-08,CN,CN,74.3,28.0,...,White,0.0,1.25173,,,NaN,118233.0,8336.0,-4.3167,-4.11991
1,2,011_S_0002,ADNI1,ADNI1,m06,2006-03-06,CN,CN,74.8,28.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-6.71646,-6.01751
2,2,011_S_0002,ADNI1,ADNI1,m36,2008-08-27,CN,CN,77.3,29.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-2.63726,-2.64089
3,2,011_S_0002,ADNIGO,ADNI1,m60,2010-09-22,CN,CN,79.3,28.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-4.16298,-5.01223
4,2,011_S_0002,ADNIGO,ADNI1,m66,2011-03-04,CN,CN,79.8,28.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-4.16298,-5.01223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786,7007,027_S_7007,ADNI3,ADNI3,bl,2021-10-18,LMCI,MCI,71.9,26.0,...,White,1.5,,,,1.3644,34064.2,5865.5,-11.1877,-7.70684
1787,7033,067_S_7033,ADNI3,ADNI3,bl,2022-02-01,AD,Dementia,59.4,26.0,...,Black,3.5,,,,NaN,21393.3,5661.3,-14.2723,-10.2599
1788,7062,021_S_7062,ADNI3,ADNI3,bl,2022-04-21,CN,CN,69.5,30.0,...,Black,0.0,,,,NaN,,,3.88952,3.86739
1789,7070,036_S_7070,ADNI3,ADNI3,bl,2022-05-04,LMCI,MCI,78.4,23.0,...,White,1.5,,,,NaN,,,-17.783,-13.6102


In [17]:
null_count(result)

RID
PTID
COLPROT
ORIGPROT
VISCODE
EXAMDATE
DX_bl
DX
AGE
MMSE
MMSE_bl
PTGENDER
PTEDUCAT
APOE4
ABETA
TAU
PTAU
ABETA_bl
TAU_bl
PTAU_bl
RAVLT_immediate_bl
RAVLT_learning_bl
RAVLT_forgetting_bl
RAVLT_perc_forgetting_bl
RAVLT_immediate
RAVLT_learning
RAVLT_forgetting
RAVLT_perc_forgetting
PTETHCAT
PTRACCAT
CDRSB
FDG
PIB
AV45
FBB
Ventricles
Hippocampus
mPACCdigit
mPACCtrailsB


In [18]:
pool1 = pd.read_csv('/Users/sohamdighe/Documents/SCIE90017/Data/ADNI/Medical_History/rec_health.csv')
pool1_set = set(pool1.RID.tolist())

In [19]:
pool2 = pd.read_csv('/Users/sohamdighe/Documents/SCIE90017/Data/ADNI/Medical_History/init_health.csv')
pool2_set = set(pool2.RID.tolist())

In [20]:
union_set = pool1_set.union(pool2_set)
rid_set = set(result.RID.tolist())
diff_set = rid_set.difference(union_set)

In [21]:
diff_set

{42, 633, 669, 1250, 4679, 4737, 4844, 4938, 6145, 6224, 6842, 6914}

In [22]:
result = result[~result.RID.isin(diff_set)] #AD cohort 266

In [23]:
unique = result.drop_duplicates(subset=['RID'], keep='last')

In [24]:
unique.columns

Index(['RID', 'PTID', 'COLPROT', 'ORIGPROT', 'VISCODE', 'EXAMDATE', 'DX_bl',
       'DX', 'AGE', 'MMSE', 'MMSE_bl', 'PTGENDER', 'PTEDUCAT', 'APOE4',
       'ABETA', 'TAU', 'PTAU', 'ABETA_bl', 'TAU_bl', 'PTAU_bl',
       'RAVLT_immediate_bl', 'RAVLT_learning_bl', 'RAVLT_forgetting_bl',
       'RAVLT_perc_forgetting_bl', 'RAVLT_immediate', 'RAVLT_learning',
       'RAVLT_forgetting', 'RAVLT_perc_forgetting', 'PTETHCAT', 'PTRACCAT',
       'CDRSB', 'FDG', 'PIB', 'AV45', 'FBB', 'Ventricles', 'Hippocampus',
       'mPACCdigit', 'mPACCtrailsB'],
      dtype='object')

In [25]:
unique.PTGENDER.value_counts()

Male      156
Female    110
Name: PTGENDER, dtype: int64

In [26]:
unique.describe()

,RID,AGE,MMSE_bl,PTEDUCAT,APOE4,TAU_bl,RAVLT_immediate_bl,RAVLT_learning_bl,RAVLT_forgetting_bl,RAVLT_perc_forgetting_bl,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting,CDRSB,FBB
count,266.000000,266.000000,265.000000,266.000000,223.000000,116.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,266.000000,19.000000
mean,4224.962406,76.181955,27.532075,15.736842,0.484305,271.268448,36.823308,4.481203,4.323308,55.624413,34.541353,4.157895,3.879699,58.443701,2.981203,1.089500
std,2366.308650,7.670865,2.537579,2.779697,0.614007,132.023179,12.798982,2.635323,3.221505,44.230610,14.661211,2.790394,3.259905,47.082746,3.912441,0.198206
min,2.000000,57.100000,16.000000,4.000000,0.000000,81.540000,8.000000,-3.000000,-28.000000,-400.000000,0.000000,-3.000000,-28.000000,-400.000000,0.000000,0.921600
25%,1567.750000,70.725000,26.000000,14.000000,0.000000,178.900000,27.000000,2.000000,3.000000,33.333300,23.000000,2.000000,2.000000,30.000000,0.000000,0.977650
50%,4781.500000,76.700000,28.000000,16.000000,0.000000,238.850000,36.000000,4.000000,4.000000,57.142900,33.000000,4.000000,4.000000,66.666700,1.500000,1.022900
75%,6459.750000,82.375000,29.000000,18.000000,1.000000,323.275000,46.000000,6.000000,6.000000,89.722225,45.000000,6.000000,6.000000,100.000000,4.000000,1.077200
max,7083.000000,93.000000,30.000000,20.000000,2.000000,851.800000,70.000000,11.000000,13.000000,100.000000,72.000000,11.000000,13.000000,100.000000,18.000000,1.659900


In [27]:
unique.groupby(['PTGENDER','DX'])[['AGE','MMSE','CDRSB','FDG','PIB','ABETA','TAU','PTAU','Ventricles','Hippocampus','mPACCdigit','mPACCtrailsB']].apply('mean')

AGE     CDRSB
PTGENDER DX                           
Female   CN        74.628846  0.278846
         Dementia  76.456522  7.391304
         MCI       74.691429  1.757143
Male     CN        75.488636  0.693182
         Dementia  77.727273  6.936364
         MCI       77.447368  2.368421

In [28]:
unique.groupby(['PTGENDER','DX'])[['AGE','MMSE','CDRSB','FDG','PIB','ABETA','TAU','PTAU','Ventricles','Hippocampus','mPACCdigit','mPACCtrailsB']].apply('mean')

AGE     CDRSB
PTGENDER DX                           
Female   CN        74.628846  0.278846
         Dementia  76.456522  7.391304
         MCI       74.691429  1.757143
Male     CN        75.488636  0.693182
         Dementia  77.727273  6.936364
         MCI       77.447368  2.368421

In [29]:
unique.DX.value_counts()

CN          96
MCI         92
Dementia    78
Name: DX, dtype: int64

In [30]:
unique.groupby('PTGENDER').DX.value_counts()

PTGENDER  DX      
Female    CN          52
          MCI         35
          Dementia    23
Male      MCI         57
          Dementia    55
          CN          44
Name: DX, dtype: int64

In [31]:
unique.groupby('PTGENDER').PTETHCAT.value_counts()

PTGENDER  PTETHCAT       
Female    Not Hisp/Latino    102
          Hisp/Latino          8
Male      Not Hisp/Latino    146
          Hisp/Latino         10
Name: PTETHCAT, dtype: int64

In [32]:
unique.groupby('PTGENDER').PTRACCAT.value_counts()

PTGENDER  PTRACCAT         
Female    White                 76
          Black                 24
          Asian                  6
          More than one          2
          Unknown                2
Male      White                131
          Black                 12
          More than one          6
          Asian                  4
          Am Indian/Alaskan      2
          Unknown                1
Name: PTRACCAT, dtype: int64

In [33]:
import csv
with open("../ADNI/selectedcolumns.csv") as file:
    tsv_file = csv.reader(file, delimiter="\t")
     
    for line in tsv_file:
        set_ptid = set(line)

In [34]:
result

,RID,PTID,COLPROT,ORIGPROT,VISCODE,EXAMDATE,DX_bl,DX,AGE,MMSE,...,PTRACCAT,CDRSB,FDG,PIB,AV45,FBB,Ventricles,Hippocampus,mPACCdigit,mPACCtrailsB
0,2,011_S_0002,ADNI1,ADNI1,bl,2005-09-08,CN,CN,74.3,28.0,...,White,0.0,1.25173,,,NaN,118233.0,8336.0,-4.3167,-4.11991
1,2,011_S_0002,ADNI1,ADNI1,m06,2006-03-06,CN,CN,74.8,28.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-6.71646,-6.01751
2,2,011_S_0002,ADNI1,ADNI1,m36,2008-08-27,CN,CN,77.3,29.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-2.63726,-2.64089
3,2,011_S_0002,ADNIGO,ADNI1,m60,2010-09-22,CN,CN,79.3,28.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-4.16298,-5.01223
4,2,011_S_0002,ADNIGO,ADNI1,m66,2011-03-04,CN,CN,79.8,28.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-4.16298,-5.01223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1786,7007,027_S_7007,ADNI3,ADNI3,bl,2021-10-18,LMCI,MCI,71.9,26.0,...,White,1.5,,,,1.3644,34064.2,5865.5,-11.1877,-7.70684
1787,7033,067_S_7033,ADNI3,ADNI3,bl,2022-02-01,AD,Dementia,59.4,26.0,...,Black,3.5,,,,NaN,21393.3,5661.3,-14.2723,-10.2599
1788,7062,021_S_7062,ADNI3,ADNI3,bl,2022-04-21,CN,CN,69.5,30.0,...,Black,0.0,,,,NaN,,,3.88952,3.86739
1789,7070,036_S_7070,ADNI3,ADNI3,bl,2022-05-04,LMCI,MCI,78.4,23.0,...,White,1.5,,,,NaN,,,-17.783,-13.6102


In [35]:
result_ptid = set(result.PTID.tolist())
final_ptid = result_ptid.intersection(set_ptid)
result = result[result.PTID.isin(final_ptid)]
result

,RID,PTID,COLPROT,ORIGPROT,VISCODE,EXAMDATE,DX_bl,DX,AGE,MMSE,...,PTRACCAT,CDRSB,FDG,PIB,AV45,FBB,Ventricles,Hippocampus,mPACCdigit,mPACCtrailsB
0,2,011_S_0002,ADNI1,ADNI1,bl,2005-09-08,CN,CN,74.3,28.0,...,White,0.0,1.25173,,,NaN,118233.0,8336.0,-4.3167,-4.11991
1,2,011_S_0002,ADNI1,ADNI1,m06,2006-03-06,CN,CN,74.8,28.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-6.71646,-6.01751
2,2,011_S_0002,ADNI1,ADNI1,m36,2008-08-27,CN,CN,77.3,29.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-2.63726,-2.64089
3,2,011_S_0002,ADNIGO,ADNI1,m60,2010-09-22,CN,CN,79.3,28.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-4.16298,-5.01223
4,2,011_S_0002,ADNIGO,ADNI1,m66,2011-03-04,CN,CN,79.8,28.0,...,White,0.0,1.33313,,,NaN,118233.0,8336.0,-4.16298,-5.01223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1215,4660,130_S_4660,ADNI2,ADNI2,bl,2012-04-30,AD,Dementia,77.2,24.0,...,Asian,3.5,1.25853,,1.5793,NaN,20511.0,4600.0,-16.6128,-15.0096
1216,4660,130_S_4660,ADNI2,ADNI2,m03,2012-08-08,AD,Dementia,77.5,24.0,...,Asian,3.5,1.25853,,,NaN,21483.0,4617.0,-16.6128,-15.0096
1217,4660,130_S_4660,ADNI2,ADNI2,m06,2012-10-17,AD,Dementia,77.7,22.0,...,Asian,3.5,1.25853,,,NaN,21483.0,4894.0,-20.4689,-15.5104
1218,4660,130_S_4660,ADNI2,ADNI2,m12,2013-05-02,AD,Dementia,78.2,19.0,...,Asian,3.5,1.25853,,,NaN,22641.0,4383.0,-22.4641,-18.9831


In [36]:
unique2  = result.drop_duplicates(subset=['RID'], keep='last')

In [37]:
unique2.PTGENDER.value_counts()

Male      57
Female    23
Name: PTGENDER, dtype: int64

In [38]:
unique2.groupby(['PTGENDER','DX'])[['AGE','MMSE','CDRSB','Ventricles','Hippocampus','mPACCdigit','mPACCtrailsB']].mean()

AGE      CDRSB
PTGENDER DX                            
Female   CN        77.000000   0.500000
         Dementia  82.671429   9.571429
         MCI       79.740000   2.000000
Male     CN        81.250000   1.375000
         Dementia  80.837500  10.062500
         MCI       81.032000   1.920000

In [39]:
unique2.groupby(['PTGENDER','DX'])[['Ventricles','Hippocampus','mPACCdigit','mPACCtrailsB']].mean()

/var/folders/1q/y31g8gyd1njfgvn3tsd2xfjr0000gn/T/ipykernel_93704/1293662604.py:1: FutureWarning: Dropping invalid columns in DataFrameGroupBy.mean is deprecated. In a future version, a TypeError will be raised. Before calling .mean, select only columns which should be valid for the function.
  unique2.groupby(['PTGENDER','DX'])[['Ventricles','Hippocampus','mPACCdigit','mPACCtrailsB']].mean()


Ventricles  mPACCdigit  mPACCtrailsB
PTGENDER DX                                              
Female   CN        30990.772727    1.052283      0.423410
         Dementia  60420.671429  -19.907214    -17.171229
         MCI       45686.420000   -6.533164     -5.987369
Male     CN        57163.693750    0.315344     -0.452769
         Dementia  57321.437500  -18.224054    -16.457219
         MCI       54721.608000   -4.924850     -4.485082

In [40]:
unique2.groupby('PTEDUCAT')[['AGE', 'MMSE','TAU','RAVLT_immediate','RAVLT_learning','RAVLT_forgetting','RAVLT_perc_forgetting']].mean()

,AGE,RAVLT_immediate,RAVLT_learning,RAVLT_forgetting,RAVLT_perc_forgetting
PTEDUCAT,,,,,
12,81.454545,24.000000,2.000000,4.181818,82.121209
13,79.000000,0.000000,0.000000,0.000000,100.000000
14,77.844444,42.666667,6.333333,5.111111,52.986267
15,81.000000,19.000000,3.000000,5.000000,100.000000
16,79.610714,35.892857,4.357143,3.750000,54.533527
17,81.116667,30.500000,4.166667,4.500000,66.730767
18,79.858333,43.583333,5.000000,4.166667,51.236264
19,81.800000,48.000000,7.000000,4.000000,33.333300
20,84.645455,34.909091,4.545455,5.090909,63.869261


In [41]:
unique2.DX.value_counts()

MCI         30
CN          27
Dementia    23
Name: DX, dtype: int64

In [42]:
unique2.groupby('PTGENDER').DX.value_counts()

PTGENDER  DX      
Female    CN          11
          Dementia     7
          MCI          5
Male      MCI         25
          CN          16
          Dementia    16
Name: DX, dtype: int64

In [43]:
unique2.groupby('PTGENDER').PTRACCAT.value_counts()

PTGENDER  PTRACCAT         
Female    White                18
          Black                 3
          Asian                 1
          More than one         1
Male      White                51
          Am Indian/Alaskan     2
          More than one         2
          Asian                 1
          Black                 1
Name: PTRACCAT, dtype: int64

In [44]:
unique2.groupby('PTGENDER').PTETHCAT.value_counts()

PTGENDER  PTETHCAT       
Female    Not Hisp/Latino    21
          Hisp/Latino         2
Male      Not Hisp/Latino    57
Name: PTETHCAT, dtype: int64

In [45]:
unique2.groupby('PTGENDER').APOE4.value_counts()

PTGENDER  APOE4
Female    0.0      15
          1.0       8
Male      0.0      34
          1.0      20
          2.0       3
Name: APOE4, dtype: int64

In [46]:
unique2.to_csv('../ADNI/diabetes.csv',index=False)

In [47]:
result_ptid = set(result.PTID.tolist())
control_ptid = set_ptid.difference(result_ptid)
print(len(result_ptid))
print(len(set_ptid))
print(len(control_ptid))

80
808
728


In [48]:
baseline = pd.read_csv('../ADNI/Data___Database/ADNIMERGE.csv',low_memory=False)
control_ptid = set(baseline.PTID.tolist()).intersection(set_ptid)
control_ptid = control_ptid.difference(result_ptid)

In [49]:
random.seed(64)
control_ids = random.sample(list(control_ptid),80)
len(control_ids)

80

In [50]:
control = baseline[baseline.PTID.isin(control_ptid)]
select_col = ['RID', 'PTID', 'EXAMDATE', 'DX', 'AGE', 'MMSE', 'CDRSB', 'FDG', 'PIB', 'AV45', 'FBB',
              'PTGENDER','PTEDUCAT','PTETHCAT','PTRACCAT','APOE4','ABETA','TAU','PTAU','Ventricles',
              'Hippocampus','mPACCdigit','mPACCtrailsB']
control = control[select_col]
# [height,weight](BMI)vitals, episodic mem, Glucose or HbA1c

In [51]:
control = control[control.PTID.isin(control_ids)]
control.EXAMDATE = pd.to_datetime(control.EXAMDATE, dayfirst=True)
control = control.sort_values(['RID', 'EXAMDATE'], ascending=[True, True])
control.columns

Index(['RID', 'PTID', 'EXAMDATE', 'DX', 'AGE', 'MMSE', 'CDRSB', 'FDG', 'PIB',
       'AV45', 'FBB', 'PTGENDER', 'PTEDUCAT', 'PTETHCAT', 'PTRACCAT', 'APOE4',
       'ABETA', 'TAU', 'PTAU', 'Ventricles', 'Hippocampus', 'mPACCdigit',
       'mPACCtrailsB'],
      dtype='object')

In [52]:
control[control.RID==205] 

,RID,PTID,EXAMDATE,DX,AGE,MMSE,CDRSB,FDG,PIB,AV45,...,PTETHCAT,PTRACCAT,APOE4,ABETA,TAU,PTAU,Ventricles,Hippocampus,mPACCdigit,mPACCtrailsB


In [53]:
collist = ['DX','AV45','MMSE','CDRSB','FDG','PIB','ABETA','TAU','PTAU','Ventricles','Hippocampus','mPACCdigit','mPACCtrailsB']
for x in collist:
    control[x] = control[x].fillna('')

In [54]:
def pullup(df,col1):
    rid_row = df.RID.tolist()
    target_row = df[col1].tolist()
    n = len(rid_row)
    for i in range(1,n):
        if len(target_row[n-i-1])==0:
            if rid_row[n-i-1] == rid_row[n-i]:
                target_row[n-i-1] = target_row[n-i]
    return target_row

control['DX'] = pullup(control,'DX')
control

,RID,PTID,EXAMDATE,DX,AGE,MMSE,CDRSB,FDG,PIB,AV45,...,PTETHCAT,PTRACCAT,APOE4,ABETA,TAU,PTAU,Ventricles,Hippocampus,mPACCdigit,mPACCtrailsB
5218,55,018_S_0055,2006-01-13,CN,75.5,29.0,0.0,1.38025,,,...,Not Hisp/Latino,White,0.0,805.2,148.7,13.39,75838.0,,-0.978323,-0.090519
105,55,018_S_0055,2006-07-18,CN,75.5,29.0,0.0,1.37992,,,...,Not Hisp/Latino,White,0.0,,,,73211.0,7560.0,-0.265272,-0.58293
5219,55,018_S_0055,2007-02-27,CN,75.5,30.0,0.0,1.41032,,,...,Not Hisp/Latino,White,0.0,,,,74959.0,,2.48273,2.45694
5220,55,018_S_0055,2007-07-06,CN,75.5,,,,,,...,Not Hisp/Latino,White,0.0,,,,,,,
5221,55,018_S_0055,2008-07-18,CN,75.5,,,,,,...,Not Hisp/Latino,White,0.0,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12567,4676,135_S_4676,2012-07-19,Dementia,78.8,,,,,,...,Not Hisp/Latino,White,1.0,,,,36391.0,5155.0,,
12568,4676,135_S_4676,2012-11-06,Dementia,78.8,26.0,3.0,,,,...,Not Hisp/Latino,White,1.0,,,,37167.0,5160.0,-8.9924,-7.21085
12569,4676,135_S_4676,2013-05-06,Dementia,78.8,28.0,1.5,,,,...,Not Hisp/Latino,White,1.0,,,,38943.0,4959.0,-7.76207,-6.25778
12570,4676,135_S_4676,2013-11-11,Dementia,78.8,,,,,,...,Not Hisp/Latino,White,1.0,,,,,,,


In [55]:
def pulldown(df,col1):
    rid_row = df.RID.tolist()
    target_row = df[col1].tolist()
    n = len(rid_row)
    for i in range(1,n):
        if target_row[i]=='':
            if rid_row[i-1] == rid_row[i]:
                target_row[i] = target_row[i-1]
    return target_row

collist = ['DX','MMSE','CDRSB','FDG','PIB','ABETA','TAU','PTAU','Ventricles','Hippocampus','mPACCdigit','mPACCtrailsB']
for x in collist:
    control[x] = pulldown(control,x)
   
control

,RID,PTID,EXAMDATE,DX,AGE,MMSE,CDRSB,FDG,PIB,AV45,...,PTETHCAT,PTRACCAT,APOE4,ABETA,TAU,PTAU,Ventricles,Hippocampus,mPACCdigit,mPACCtrailsB
5218,55,018_S_0055,2006-01-13,CN,75.5,29.0,0.0,1.38025,,,...,Not Hisp/Latino,White,0.0,805.2,148.7,13.39,75838.0,,-0.978323,-0.090519
105,55,018_S_0055,2006-07-18,CN,75.5,29.0,0.0,1.37992,,,...,Not Hisp/Latino,White,0.0,805.2,148.7,13.39,73211.0,7560.0,-0.265272,-0.582930
5219,55,018_S_0055,2007-02-27,CN,75.5,30.0,0.0,1.41032,,,...,Not Hisp/Latino,White,0.0,805.2,148.7,13.39,74959.0,7560.0,2.482730,2.456940
5220,55,018_S_0055,2007-07-06,CN,75.5,30.0,0.0,1.41032,,,...,Not Hisp/Latino,White,0.0,805.2,148.7,13.39,74959.0,7560.0,2.482730,2.456940
5221,55,018_S_0055,2008-07-18,CN,75.5,30.0,0.0,1.41032,,,...,Not Hisp/Latino,White,0.0,805.2,148.7,13.39,74959.0,7560.0,2.482730,2.456940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12567,4676,135_S_4676,2012-07-19,Dementia,78.8,25.0,3.5,1.32544,,,...,Not Hisp/Latino,White,1.0,>1700,450.9,39.14,36391.0,5155.0,-9.273340,-7.756580
12568,4676,135_S_4676,2012-11-06,Dementia,78.8,26.0,3.0,1.32544,,,...,Not Hisp/Latino,White,1.0,>1700,450.9,39.14,37167.0,5160.0,-8.992400,-7.210850
12569,4676,135_S_4676,2013-05-06,Dementia,78.8,28.0,1.5,1.32544,,,...,Not Hisp/Latino,White,1.0,>1700,450.9,39.14,38943.0,4959.0,-7.762070,-6.257780
12570,4676,135_S_4676,2013-11-11,Dementia,78.8,28.0,1.5,1.32544,,,...,Not Hisp/Latino,White,1.0,>1700,450.9,39.14,38943.0,4959.0,-7.762070,-6.257780


In [56]:
null_count(control)

RID 0
PTID 0
EXAMDATE 0
DX 0
AGE 0
MMSE 0
CDRSB 0
FDG 161
PIB 850
AV45 667
FBB 0
PTGENDER 0
PTEDUCAT 0
PTETHCAT 0
PTRACCAT 0
APOE4 0
ABETA 217
TAU 217
PTAU 217
Ventricles 11
Hippocampus 17
mPACCdigit 0
mPACCtrailsB 0


In [57]:
control_rid = control.RID.tolist()
control

,RID,PTID,EXAMDATE,DX,AGE,MMSE,CDRSB,FDG,PIB,AV45,...,PTETHCAT,PTRACCAT,APOE4,ABETA,TAU,PTAU,Ventricles,Hippocampus,mPACCdigit,mPACCtrailsB
5218,55,018_S_0055,2006-01-13,CN,75.5,29.0,0.0,1.38025,,,...,Not Hisp/Latino,White,0.0,805.2,148.7,13.39,75838.0,,-0.978323,-0.090519
105,55,018_S_0055,2006-07-18,CN,75.5,29.0,0.0,1.37992,,,...,Not Hisp/Latino,White,0.0,805.2,148.7,13.39,73211.0,7560.0,-0.265272,-0.582930
5219,55,018_S_0055,2007-02-27,CN,75.5,30.0,0.0,1.41032,,,...,Not Hisp/Latino,White,0.0,805.2,148.7,13.39,74959.0,7560.0,2.482730,2.456940
5220,55,018_S_0055,2007-07-06,CN,75.5,30.0,0.0,1.41032,,,...,Not Hisp/Latino,White,0.0,805.2,148.7,13.39,74959.0,7560.0,2.482730,2.456940
5221,55,018_S_0055,2008-07-18,CN,75.5,30.0,0.0,1.41032,,,...,Not Hisp/Latino,White,0.0,805.2,148.7,13.39,74959.0,7560.0,2.482730,2.456940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12567,4676,135_S_4676,2012-07-19,Dementia,78.8,25.0,3.5,1.32544,,,...,Not Hisp/Latino,White,1.0,>1700,450.9,39.14,36391.0,5155.0,-9.273340,-7.756580
12568,4676,135_S_4676,2012-11-06,Dementia,78.8,26.0,3.0,1.32544,,,...,Not Hisp/Latino,White,1.0,>1700,450.9,39.14,37167.0,5160.0,-8.992400,-7.210850
12569,4676,135_S_4676,2013-05-06,Dementia,78.8,28.0,1.5,1.32544,,,...,Not Hisp/Latino,White,1.0,>1700,450.9,39.14,38943.0,4959.0,-7.762070,-6.257780
12570,4676,135_S_4676,2013-11-11,Dementia,78.8,28.0,1.5,1.32544,,,...,Not Hisp/Latino,White,1.0,>1700,450.9,39.14,38943.0,4959.0,-7.762070,-6.257780


In [58]:
for id in control_rid:
    eg = control[control.RID==id]
    for i in range(len(eg)):
        diff = eg.EXAMDATE.iloc[i] - eg.EXAMDATE.iloc[0]
        eg['AGE'].iloc[i] =round(diff.days/365,1) + eg.AGE.iloc[0]
    control[control.RID==id] = eg

/var/folders/1q/y31g8gyd1njfgvn3tsd2xfjr0000gn/T/ipykernel_93704/1774211902.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  eg['AGE'].iloc[i] =round(diff.days/365,1) + eg.AGE.iloc[0]


In [59]:
unique3 = control.drop_duplicates(subset=['RID'], keep='last')
unique3

,RID,PTID,EXAMDATE,DX,AGE,MMSE,CDRSB,FDG,PIB,AV45,...,PTETHCAT,PTRACCAT,APOE4,ABETA,TAU,PTAU,Ventricles,Hippocampus,mPACCdigit,mPACCtrailsB
11116,55,018_S_0055,2015-07-06,MCI,85.0,28.0,2.0,1.27504,,,...,Not Hisp/Latino,White,0.0,805.2,148.7,13.39,81910.0,7560.0,1.149120,-1.435860
11089,61,023_S_0061,2016-02-18,Dementia,87.2,24.0,3.5,1.1977,,,...,Not Hisp/Latino,Black,0.0,837.3,322.7,30.27,37547.0,7071.0,-16.612800,-15.031200
13078,69,100_S_0069,2020-01-23,MCI,86.9,28.0,0.5,1.28195,,1.0375,...,Not Hisp/Latino,White,0.0,,,,52164.4,7354.6,-1.878180,-0.504887
15867,89,073_S_0089,2021-08-11,CN,80.6,30.0,0.0,1.2886,,,...,Not Hisp/Latino,White,0.0,1604,171.3,15.77,23991.0,7479.0,2.061920,2.961590
11006,126,023_S_0126,2016-03-25,Dementia,87.6,16.0,9.0,,,,...,Not Hisp/Latino,White,1.0,614.6,289.8,22.34,55222.0,3628.0,-25.974800,-22.781700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4469,4635,116_S_4635,2016-05-16,MCI,76.8,30.0,0.0,1.34822,,1.5333,...,Not Hisp/Latino,White,1.0,324.9,214,21.94,13895.0,7115.0,0.405032,-0.475104
13773,4643,012_S_4643,2019-09-10,CN,72.5,30.0,0.0,1.35019,,1.0159,...,Not Hisp/Latino,White,0.0,>1700,227.9,19.42,45697.4,7184.0,3.147020,3.406310
4407,4657,135_S_4657,2014-04-28,Dementia,73.5,11.0,14.0,1.05223,,1.4917,...,Not Hisp/Latino,White,1.0,883.8,648.4,59.14,49222.0,4410.0,-31.826100,-27.170200
4371,4672,137_S_4672,2015-10-26,Dementia,70.3,16.0,3.0,0.96936,,,...,Not Hisp/Latino,White,2.0,497,367.2,36.27,44989.0,5403.0,-25.974800,-25.974800


In [60]:
unique3.to_csv('../ADNI/control.csv',index = False)

In [61]:
unique3.groupby(['PTGENDER','DX'])[['AGE','MMSE','CDRSB','FDG','PIB','ABETA','TAU','PTAU','Ventricles','Hippocampus','mPACCdigit','mPACCtrailsB']].apply('mean')

AGE       MMSE     CDRSB  mPACCdigit  mPACCtrailsB
PTGENDER DX                                                                
Female   CN        79.283333  28.583333  0.250000    0.399088     -0.208131
         Dementia  80.591667  17.500000  9.333333  -24.775208    -23.074567
         MCI       77.150000  28.000000  1.333333   -4.063299     -3.461387
Male     CN        80.708333  28.500000  0.166667   -1.469229     -1.333625
         Dementia  80.872727  21.636364  6.954545  -17.192582    -15.106550
         MCI       81.161905  27.761905  0.976190   -4.403749     -4.242128

In [62]:
unique3.PTGENDER.value_counts()

Male      44
Female    36
Name: PTGENDER, dtype: int64

In [63]:
unique3.groupby(['PTGENDER']).DX.value_counts()

PTGENDER  DX      
Female    CN          12
          Dementia    12
          MCI         12
Male      MCI         21
          CN          12
          Dementia    11
Name: DX, dtype: int64

In [64]:
unique3.DX.value_counts()

MCI         33
CN          24
Dementia    23
Name: DX, dtype: int64

In [65]:
unique3.to_csv('../ADNI/control.csv',index=False)

In [66]:
unique3

,RID,PTID,EXAMDATE,DX,AGE,MMSE,CDRSB,FDG,PIB,AV45,...,PTETHCAT,PTRACCAT,APOE4,ABETA,TAU,PTAU,Ventricles,Hippocampus,mPACCdigit,mPACCtrailsB
11116,55,018_S_0055,2015-07-06,MCI,85.0,28.0,2.0,1.27504,,,...,Not Hisp/Latino,White,0.0,805.2,148.7,13.39,81910.0,7560.0,1.149120,-1.435860
11089,61,023_S_0061,2016-02-18,Dementia,87.2,24.0,3.5,1.1977,,,...,Not Hisp/Latino,Black,0.0,837.3,322.7,30.27,37547.0,7071.0,-16.612800,-15.031200
13078,69,100_S_0069,2020-01-23,MCI,86.9,28.0,0.5,1.28195,,1.0375,...,Not Hisp/Latino,White,0.0,,,,52164.4,7354.6,-1.878180,-0.504887
15867,89,073_S_0089,2021-08-11,CN,80.6,30.0,0.0,1.2886,,,...,Not Hisp/Latino,White,0.0,1604,171.3,15.77,23991.0,7479.0,2.061920,2.961590
11006,126,023_S_0126,2016-03-25,Dementia,87.6,16.0,9.0,,,,...,Not Hisp/Latino,White,1.0,614.6,289.8,22.34,55222.0,3628.0,-25.974800,-22.781700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4469,4635,116_S_4635,2016-05-16,MCI,76.8,30.0,0.0,1.34822,,1.5333,...,Not Hisp/Latino,White,1.0,324.9,214,21.94,13895.0,7115.0,0.405032,-0.475104
13773,4643,012_S_4643,2019-09-10,CN,72.5,30.0,0.0,1.35019,,1.0159,...,Not Hisp/Latino,White,0.0,>1700,227.9,19.42,45697.4,7184.0,3.147020,3.406310
4407,4657,135_S_4657,2014-04-28,Dementia,73.5,11.0,14.0,1.05223,,1.4917,...,Not Hisp/Latino,White,1.0,883.8,648.4,59.14,49222.0,4410.0,-31.826100,-27.170200
4371,4672,137_S_4672,2015-10-26,Dementia,70.3,16.0,3.0,0.96936,,,...,Not Hisp/Latino,White,2.0,497,367.2,36.27,44989.0,5403.0,-25.974800,-25.974800


In [67]:
u2 = unique2[unique3.columns]
u2.to_csv('../ADNI/diabetes.csv',index=False)